In [47]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplcursors # Use this is for creating a cursor-interactive plot with "%matplotlib notebook"
from sklearn.decomposition import NMF # Use this for training Non-negative Matrix Factorization
from sklearn.utils.extmath import randomized_svd # Use this for training Singular Value Decomposition
from sklearn.manifold import TSNE # Use this for training t-sne manifolding

In [48]:
dir = './최종데이터/'
#df_product = pd.read_csv(dir + 'basic_data_edit.csv', usecols=['00.상품_URL','00.상품코드','01.브랜드','02.상품명','03.가격','04.제품 주요 사양','05.모든 성분','06.총 평점','07.리뷰 개수','08_1.별점 1점','08_2.별점 2점','08_3.별점 3점','08_4.별점 4점','08_5.별점 5점','09_1.피부타입_건성','09_2.피부타입_복합성','09_3.피부타입_지성','10_1.피부고민_보습','10_2.피부고민_진정','10_3.피부고민_주름/미백','11_1.피부자극_없음','11_2.피부자극_보통','11_3.피부자극_있음'],encoding='cp949')
df_product = pd.read_csv(dir + 'basic_data_img.csv', usecols=['00.상품코드','00.상품_URL','00.이미지_URL','01.브랜드','02.상품명','03.가격','04.제품 주요 사양','05.모든 성분','06.총 평점','07.리뷰 개수','08_1.별점 1점','08_2.별점 2점','08_3.별점 3점','08_4.별점 4점','08_5.별점 5점','09_1.피부타입_건성','09_2.피부타입_복합성','09_3.피부타입_지성','10_1.피부고민_보습','10_2.피부고민_진정','10_3.피부고민_주름/미백','11_1.피부자극_없음','11_2.피부자극_보통','11_3.피부자극_있음'],encoding='cp949')
df_review = pd.read_csv(dir + 'total_review.csv', usecols=['code','user','type','tone','problem','rating','feature','review','total_rating'],encoding='cp949')
print('done')

done


In [49]:
df_review

,code,user,type,tone,problem,rating,feature,review,total_rating
0,A000000147339,요다,건성,웜톤,민감성 | 아토피 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",개인적으로 성분으로 마케팅하는 제품들을 그렇게 좋아하진 않습니다. 특히 토너는 순한...,5.0
1,A000000147339,올리브양,복합성,쿨톤,모공 | 민감성 | 주름 | 탄력 | 트러블 | 홍조,5.0,"('건성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",복합성 피부에 많이 민감한 피부입니다. 요즘 봄 바람에 아침 저녁으로 기온차가 심해...,5.0
2,A000000147339,뚜비이,건성,쿨톤,민감성 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",지난번 올영체험단으로 제공받아 사용해보고 너무 좋아서 다시 구매했습니당!\r\n저는...,5.0
3,A000000147339,시래우,복합성,겨울쿨톤,다크서클 | 모공 | 미백 | 민감성 | 블랙헤드 | 아토피 | 잡티 | 주름 | ...,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",날씨 더워지면 광대가 뜨거워지면서 내 얼굴이 호빵맨이 되어가는 것 같고 좀 더 더워...,5.0
4,A000000147339,효지니,복합성,쿨톤,민감성 | 블랙헤드 | 아토피 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",요즘에 피부가 너무 나빠져서 잘 맞는 화장품 찾아볼려고 이것저것 구매를 해봤어요! ...,5.0
...,...,...,...,...,...,...,...,...,...
16500,A000000148095,겨울별,지성,쿨톤,각질 | 미백 | 블랙헤드 | 잡티 | 주름 | 트러블,5.0,"('지성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",봄철에딱인 속건조토너 흑미탄탄토너를 일주일정도 써보았습니다. 뷰티유튜버가 왜추천을했...,3.5
16501,A000000148095,moon219,복합성,쿨톤,모공 | 트러블,5.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",제품이 순하기도 하고 성분이 좋아서 그런지 토너팩을 매일같이 하니까 정말 많이 진정...,5.0
16502,A000000148095,리파리해파리파리,건성,웜톤,각질 | 모공 | 미백 | 블랙헤드 | 잡티 | 주름 | 탄력 | 트러블,4.0,"('복합성에 좋아요 | 주름/미백에 좋아요 | 자극없이 순해요',)",저는 올리브영체험단을 몇번하면서\r\n항상 제품과 리뷰작성바란다는 안내문만 받아보다...,3.0
16503,A000000148095,luv9,건성,웜톤,각질 | 민감성 | 주름 | 홍조,4.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",체험단 신청해서 받아보게 되었는데! 직접 손편지로 리뷰작성안내 해주신 브랜드는 처음...,3.5


In [50]:
#2개 이상의 리뷰를 단 유저들만 추출
user_review_count = df_review['user'].value_counts()
user_review_count = pd.DataFrame(user_review_count)
user_review_count = user_review_count.reset_index()
user_review_count.columns = ['user','count']

df_review_count = pd.merge(df_review,user_review_count,on='user',how='left')
df_review_count = df_review_count[df_review_count['count']>=2]
df_review_count

,code,user,type,tone,problem,rating,feature,review,total_rating,count
0,A000000147339,요다,건성,웜톤,민감성 | 아토피 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",개인적으로 성분으로 마케팅하는 제품들을 그렇게 좋아하진 않습니다. 특히 토너는 순한...,5.0,14
1,A000000147339,올리브양,복합성,쿨톤,모공 | 민감성 | 주름 | 탄력 | 트러블 | 홍조,5.0,"('건성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",복합성 피부에 많이 민감한 피부입니다. 요즘 봄 바람에 아침 저녁으로 기온차가 심해...,5.0,13
2,A000000147339,뚜비이,건성,쿨톤,민감성 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",지난번 올영체험단으로 제공받아 사용해보고 너무 좋아서 다시 구매했습니당!\r\n저는...,5.0,6
3,A000000147339,시래우,복합성,겨울쿨톤,다크서클 | 모공 | 미백 | 민감성 | 블랙헤드 | 아토피 | 잡티 | 주름 | ...,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",날씨 더워지면 광대가 뜨거워지면서 내 얼굴이 호빵맨이 되어가는 것 같고 좀 더 더워...,5.0,13
4,A000000147339,효지니,복합성,쿨톤,민감성 | 블랙헤드 | 아토피 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",요즘에 피부가 너무 나빠져서 잘 맞는 화장품 찾아볼려고 이것저것 구매를 해봤어요! ...,5.0,10
...,...,...,...,...,...,...,...,...,...,...
16500,A000000148095,겨울별,지성,쿨톤,각질 | 미백 | 블랙헤드 | 잡티 | 주름 | 트러블,5.0,"('지성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",봄철에딱인 속건조토너 흑미탄탄토너를 일주일정도 써보았습니다. 뷰티유튜버가 왜추천을했...,3.5,5
16501,A000000148095,moon219,복합성,쿨톤,모공 | 트러블,5.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",제품이 순하기도 하고 성분이 좋아서 그런지 토너팩을 매일같이 하니까 정말 많이 진정...,5.0,6
16502,A000000148095,리파리해파리파리,건성,웜톤,각질 | 모공 | 미백 | 블랙헤드 | 잡티 | 주름 | 탄력 | 트러블,4.0,"('복합성에 좋아요 | 주름/미백에 좋아요 | 자극없이 순해요',)",저는 올리브영체험단을 몇번하면서\r\n항상 제품과 리뷰작성바란다는 안내문만 받아보다...,3.0,27
16503,A000000148095,luv9,건성,웜톤,각질 | 민감성 | 주름 | 홍조,4.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",체험단 신청해서 받아보게 되었는데! 직접 손편지로 리뷰작성안내 해주신 브랜드는 처음...,3.5,10


In [51]:
#사용자가 입력한 속성과 가장 비슷한 기존 유저를 뽑는 작업
df_user_feature = df_review_count[['user','type','tone','problem']]
df_user_feature = df_user_feature.drop_duplicates(['user'])

#user의 통합 feature를 담을 데이터 프레임
df_feature = pd.DataFrame(columns = ['user','feature'])

for index in df_user_feature.index:
    user = df_user_feature.loc[index]['user']
    problems = df_user_feature.loc[index]['problem']
    problems_list = problems.split('|')
    
    problem = ''
    for i in problems_list:
        i = i.replace(" ","")
        problem += (i+' ')
    
    feature = str(df_user_feature.loc[index]['type']) +' '+str(df_user_feature.loc[index]['tone']) + ' '+problem
    
    df_feature.loc[str(index)] = [user,feature]
    
#사용자가 입력한 정보를 df_feature 맨 아래에 추가  
df_feature
df_feature.loc[str(index)] = ('input','복합성 쿨톤 모공 민감성 주름 탄력 트러블 홍조')
df_feature = df_feature.reset_index(drop=True)
print(df_feature)

        user                                            feature
0         요다                                 건성 웜톤 민감성 아토피 트러블 
1       올리브양                        복합성 쿨톤 모공 민감성 주름 탄력 트러블 홍조 
2        뚜비이                                  건성 쿨톤 민감성 잡티 트러블 
3        시래우  복합성 겨울쿨톤 다크서클 모공 미백 민감성 블랙헤드 아토피 잡티 주름 탄력 트러블 홍조 
4        효지니                        복합성 쿨톤 민감성 블랙헤드 아토피 잡티 트러블 
...      ...                                                ...
3161  dawoon                                    건성 웜톤 모공 잡티 탄력 
3162   고구맛탱이                                        건성 쿨톤 블랙헤드 
3163    에주에주                                       지성 웜톤 모공 미백 
3164   grape        복합성 쿨톤 각질 모공 미백 민감성 블랙헤드 잡티 주름 탄력 트러블 피지과다 
3165   input                         복합성 쿨톤 모공 민감성 주름 탄력 트러블 홍조

[3166 rows x 2 columns]


In [52]:
from sklearn.feature_extraction.text import CountVectorizer

counter_vector = CountVectorizer(ngram_range=(1,3))
c_vector_features = counter_vector.fit_transform(df_feature['feature'])
c_vector_features.shape

(3166, 883)

In [53]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_feature = cosine_similarity(c_vector_features,c_vector_features).argsort()[:,::-1]
print(similarity_feature)
print(similarity_feature.shape)

[[   0  416 2799 ... 2271 2259 1582]
 [3165    1 1361 ... 1797  734 2963]
 [ 733 1717    2 ...  276 1478 1633]
 ...
 [ 817 3163 1951 ...  788 2510 2144]
 [1550 3164 2612 ...  980  734 2855]
 [3165    1 1361 ... 1797  734 2963]]
(3166, 3166)


In [54]:
def recommend_user_list(df_feature, user , top=3):
    #특정 제품코드 뽑아내기
    target_feature_index = df_feature[df_feature['user'] == user].index.values

    #타켓제품과 비슷한 코사인 유사도값
    sim_index = similarity_feature[int(target_feature_index),:top].reshape(-1)
    #본인제외
    sim_index = sim_index[sim_index!=target_feature_index]
    
    #추천 결과 새로운 df 생성, 평균평점(score)으로 정렬
    result = df_feature.iloc[sim_index]
    
    return result

In [55]:
df_result = recommend_user_list(df_feature, user='input')
similar_user = df_result['user']
similar_user = similar_user.reset_index(drop=True)
similar_user = similar_user[0]
similar_user

'올리브양'

In [56]:
# 고유 사용자, 고유 제품 갯수 확인
n_users = len(df_review_count['user'].unique())
n_movies = len(df_review_count['code'].unique())

n_users, n_movies # 610 명의 사용자가 9724개의 영화에 평점을 매김을 확인

(3166, 368)

In [57]:
A = df_review_count.pivot_table(index = 'code', columns = 'user',values = 'total_rating')
A = A.copy().fillna(0)
print(A)

user           02k  0321  0eun   0교   11  11월의온조  12최정  16살  17호빋베브봄  1등급  \
code                                                                        
A000000002839  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000004434  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000005060  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000005766  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000005767  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
...            ...   ...   ...  ...  ...     ...   ...  ...      ...  ...   
A000000163655  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000163896  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000164211  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000165599  0.0   0.0   0.0  0.0  0.0     0.0   0.0  0.0      0.0  0.0   
A000000166043  0.0   0.0   0.0  0.0  4.0     0.0   0.0  0.0      0.0  0.0   

In [58]:
final_df = df_review_count[['user','code','total_rating']]
final_df

,user,code,total_rating
0,요다,A000000147339,5.0
1,올리브양,A000000147339,5.0
2,뚜비이,A000000147339,5.0
3,시래우,A000000147339,5.0
4,효지니,A000000147339,5.0
...,...,...,...
16500,겨울별,A000000148095,3.5
16501,moon219,A000000148095,5.0
16502,리파리해파리파리,A000000148095,3.0
16503,luv9,A000000148095,3.5


In [59]:
import surprise
reader = surprise.Reader(rating_scale = (1,5))

col_list = ['user','code','total_rating']
data = surprise.Dataset.load_from_df(final_df[col_list], reader)

In [60]:
#surprise KNN 기반 CF 진행

#학습
trainset = data.build_full_trainset()
option = {'name' : 'pearson'}
algo = surprise.KNNBasic(sim_options = option)

algo.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [61]:
name_list = final_df['user'].unique()
name_list = pd.Series(name_list)

index = name_list[name_list == similar_user].index[0]
index

1

In [62]:
name_list = final_df['user'].unique()
name_list = pd.Series(name_list)
name_list

0           요다
1         올리브양
2          뚜비이
3          시래우
4          효지니
         ...  
3161    dawoon
3162     고구맛탱이
3163      에주에주
3164     grape
3165       겨울별
Length: 3166, dtype: object

In [63]:
result = algo.get_neighbors(index,k=5)
print('해당 사용자와 가장 유사한 사용자들은?', result)
print(name_list[result])

해당 사용자와 가장 유사한 사용자들은? [3, 11, 12, 54, 88]
3            시래우
11           핫태맘
12    Alliveholy
54          우와아앙
88     우리집고양이이쁘죠
dtype: object


In [64]:
code_list = []
for r1 in result:
    max_rating = data.df[data.df['user']==name_list[r1]]['total_rating'].max()
    cos_id = data.df[(data.df['total_rating']==max_rating)&(data.df['user']==name_list[r1])]['code'].values
    
    code_list.append(cos_id)
code_list

[array(['A000000147339', 'A000000166043', 'A000000129771'], dtype=object),
 array(['A000000147339', 'A000000145578', 'A000000143827', 'A000000159657',
        'A000000160562', 'A000000154540', 'A000000104169', 'A000000143195',
        'A000000153886', 'A000000153886'], dtype=object),
 array(['A000000147339'], dtype=object),
 array(['A000000147339', 'A000000141704', 'A000000162662', 'A000000162662',
        'A000000113008', 'A000000151890', 'A000000151890'], dtype=object),
 array(['A000000147339', 'A000000147339', 'A000000147339', 'A000000147339',
        'A000000148391', 'A000000148391', 'A000000148391', 'A000000148391',
        'A000000148391', 'A000000148391', 'A000000148391', 'A000000148391',
        'A000000148391', 'A000000148391'], dtype=object)]

In [65]:
df_product

,00.상품코드,00.상품_URL,00.이미지_URL,01.브랜드,02.상품명,03.가격,04.제품 주요 사양,05.모든 성분,06.총 평점,07.리뷰 개수,...,08_5.별점 5점,09_1.피부타입_건성,09_2.피부타입_복합성,09_3.피부타입_지성,10_1.피부고민_보습,10_2.피부고민_진정,10_3.피부고민_주름/미백,11_1.피부자극_없음,11_2.피부자극_보통,11_3.피부자극_있음
0,A000000147339,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,아누아,[찐진정 토너] 아누아 어성초 77% 토너 350ml,30500,모든 피부용,약모밀추출물 정제수 12헥산다이올 글리세린 베타인 병풀추출물 체이스트트리추출물 우엉...,4.7,6949,...,0.80,0.18,0.63,0.19,0.17,0.83,0.01,0.76,0.22,0.02
1,A000000137180,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,라운드랩,[증량] 라운드랩 1025 독도 토너 (본품500ml+100ml 추가 증정),30000,모든 피부용,정제수 부틸렌글라이콜 글리세린 펜틸렌글라이콜 프로판다이올 아이리쉬모스추출물 사탕수수...,4.8,5993,...,0.84,0.22,0.63,0.15,0.39,0.61,0.01,0.82,0.18,0.01
2,A000000150946,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,넘버즈인,넘버즈인 2번 메이크업 찰떡 토너 200ml,22000,모든 피부 용,온천수 12헥산다이올 메틸프로판다이올 글리세린 나이아신아마이드 솔비톨 부틸렌글라이콜...,4.6,1748,...,0.74,0.35,0.56,0.09,0.76,0.19,0.05,0.60,0.35,0.05
3,A000000145578,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,닥터지,닥터지 레드 블레미쉬 포 맨 올인원 플루이드 (본품150ml+30ml 추가 증정),29000,민감 피부 트러블 피부,정제수 나이아신아마이드 아이소도데케인 카프릴릴메티콘 메틸프로판다이올 12헥산다이올 ...,4.7,3954,...,0.79,0.16,0.65,0.19,0.39,0.59,0.02,0.67,0.33,0.01
4,A000000150722,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,한율,한율 어린쑥 수분진정 2종세트(크림 15ml증정),49000,모든피부,토너 정제수 프로판다이올 글리세린 12헥산다이올 폴리글리세릴6카프릴레이트 폴리글리세...,4.8,623,...,0.84,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,A000000142618,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,크리니크,크리니크 포맨 맥시멈 하이드레이터 72-아워 오토-리플레니싱 하이드레이터,41000,모든 피부 타입,정제수다이메티콘부틸렌글라이콜글리세린트라이실록세인트레할로오스수크로오스암모늄아크릴로일다...,4.8,43,...,0.79,0.19,0.49,0.33,0.77,0.21,0.02,0.67,0.30,0.03
516,A000000142413,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,멜릭서,멜릭서 비건 밸런싱 토너 듀오세트 150ml+150ml,18000,극 민감성부터 지복합성까지 모든 피부에 추천합니다,녹차추출물 100,4.7,392,...,0.76,0.17,0.58,0.25,0.24,0.76,0.01,0.82,0.16,0.02
517,A000000142412,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,멜릭서,멜릭서 비건 밸런싱 토너 150ml,12000,극 민감성부터 지복합성까지 모든 피부에 추천합니다,녹차추출물 100,4.7,75,...,0.72,0.11,0.60,0.29,0.21,0.77,0.02,0.88,0.12,0.01
518,A000000142256,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,AHC,AHC 리얼 너리싱 스페셜 케어,34900,모든 피부용,AHC 리얼 너리싱 세럼정제수 글리세린 프로판다이올 글리세레스26 나이아신아마이드 ...,4.7,16,...,0.75,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [66]:
result_dict={}
products_dict = {}
i = 0
for codes in code_list:
    for code in codes:
#         print(df_product[df_product['00.상품코드']==code]['00.상품_URL'].item())
#         print(df_product[df_product['00.상품코드']==code]['02.상품명'].item())
        
        product_dict = {}
        product_dict['productURL'] = str(df_product[df_product['00.상품코드']==code]['00.상품_URL'].item())
        product_dict['imageURL'] = str(df_product[df_product['00.상품코드']==code]['00.이미지_URL'].item())
        product_dict['brand'] = str(df_product[df_product['00.상품코드']==code]['01.브랜드'].item())
        product_dict['productName'] = str(df_product[df_product['00.상품코드']==code]['02.상품명'].item())
        product_dict['price'] = int(df_product[df_product['00.상품코드']==code]['03.가격'].item())
        
        products_dict[i] = product_dict
        i+=1
        if i==10:
            break
    if i==10:
        break
        
result_dict['CF'] = products_dict
print(result_dict)

{'CF': {0: {'productURL': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000147339&dispCatNo=100000100010008&trackingCd=Cat100000100010008_MID&curation&egcode&rccode&egrankcode', 'imageURL': 'https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0014/A00000014733907ko.jpg?l=ko', 'brand': '아누아', 'productName': '[찐진정 토너] 아누아 어성초 77% 토너 350ml', 'price': 30500}, 1: {'productURL': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000166043&dispCatNo=100000100010008&trackingCd=Cat100000100010008_MID&curation&egcode&rccode&egrankcode', 'imageURL': 'https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0016/A00000016604301ko.jpg?l=ko', 'brand': '피지오겔', 'productName': '[APP 첫구매딜]피지오겔 레드수딩AI 로션 50ml', 'price': 100}, 2: {'productURL': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000129771&dispCatNo=100000100010008&trackingCd=Cat100000100010008_MID&curation&egcode&rccode&egrankcode', 'imageURL': 'https://